In [2]:
import requests
import json
import pandas as pd

### Load fresh data

In [16]:
# url = "https://bdfed.stitch.mlbinfra.com/bdfed/stats/player?stitch_env=prod&season=2021&sportId=1&stats=season&group=hitting&gameType=R&limit=25&offset=0&sortStat=runsBattedIn&order=desc"

# r = requests.get(url)

# player_data = r.json()

# df = pd.json_normalize(player_data['stats'])
# print(df.head())

   year  playerId         playerName    type  rank     playerFullName  \
0  2021    521692     Salvador Perez  player     1     Salvador Perez   
1  2021    547989         Jose Abreu  player     2         Jose Abreu   
2  2021    606192  Teoscar Hernandez  player     3  Teoscar Hernandez   
3  2021    646240      Rafael Devers  player     4      Rafael Devers   
4  2021    594807        Adam Duvall  player     4        Adam Duvall   

  playerFirstName playerLastName playerUseName playerInitLastName  ...   slg  \
0        Salvador          Perez      Salvador            S Perez  ...  .544   
1            Jose          Abreu          Jose            J Abreu  ...  .481   
2         Teoscar      Hernandez       Teoscar        T Hernández  ...  .524   
3          Rafael         Devers        Rafael           R Devers  ...  .538   
4            Adam         Duvall          Adam           A Duvall  ...  .491   

    ops caughtStealing stolenBases stolenBasePercentage  groundIntoDoublePlay  \

### Load saved file

In [46]:
df = pd.read_csv("2021_mlb_data.csv")
df.head()

,year,playerId,playerName,type,rank,playerFullName,playerFirstName,playerLastName,playerUseName,playerInitLastName,...,slg,ops,caughtStealing,stolenBases,stolenBasePercentage,groundIntoDoublePlay,rbi,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun
0,2021,521692,Salvador Perez,player,1,Salvador Perez,Salvador,Perez,Salvador,S Perez,...,0.544,0.860,0,1,1.000,14,121,0.86,0,12.92
1,2021,547989,Jose Abreu,player,2,Jose Abreu,Jose,Abreu,Jose,J Abreu,...,0.481,0.832,0,1,1.000,28,117,1.21,0,18.87
2,2021,606192,Teoscar Hernandez,player,3,Teoscar Hernandez,Teoscar,Hernandez,Teoscar,T Hernández,...,0.524,0.870,4,12,.750,5,116,0.84,0,17.19
3,2021,646240,Rafael Devers,player,4,Rafael Devers,Rafael,Devers,Rafael,R Devers,...,0.538,0.890,5,5,.500,13,113,0.99,0,15.55
4,2021,594807,Adam Duvall,player,4,Adam Duvall,Adam,Duvall,Adam,A Duvall,...,0.491,0.772,0,5,1.000,7,113,0.57,0,13.50


## Remove Columns

In [47]:
drop_cols = ["year", "playerName", "teamAbbrev", "position", "playerId", "type", "rank", "playerFullName", "playerFirstName",
             "playerLastName", "playerUseName", "playerInitLastName", "teamId", "teamName","teamShortName",
             "leagueName","leagueId","positionAbbrev", "primaryPositionAbbrev"]

In [48]:
df.drop(columns=drop_cols, axis=1, inplace=True)
df.head()

,plateAppearances,totalBases,leftOnBase,sacBunts,sacFlies,babip,extraBaseHits,hitByPitch,gidp,gidpOpp,...,slg,ops,caughtStealing,stolenBases,stolenBasePercentage,groundIntoDoublePlay,rbi,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun
0,665,337,278,0,4,0.298,72,13,14,118,...,0.544,0.860,0,1,1.000,14,121,0.86,0,12.92
1,659,272,249,0,10,0.293,62,22,28,113,...,0.481,0.832,0,1,1.000,28,117,1.21,0,18.87
2,595,288,227,0,2,0.352,61,7,5,92,...,0.524,0.870,4,12,.750,5,116,0.84,0,17.19
3,664,318,284,0,4,0.307,76,7,13,95,...,0.538,0.890,5,5,.500,13,113,0.99,0,15.55
4,555,252,233,0,3,0.260,57,4,7,68,...,0.491,0.772,0,5,1.000,7,113,0.57,0,13.50


In [55]:
bases_mean = df[df["stolenBasePercentage"] != ".---"]

In [56]:
bases_mean = pd.to_numeric(bases_mean["stolenBasePercentage"])
bases_mean.mean()

0.824125

In [57]:
df.replace({"stolenBasePercentage": {".---" : bases_mean.mean()}}, inplace=True)

In [58]:
df["stolenBasePercentage"]

0        1.000
1        1.000
2         .750
3         .500
4        1.000
5         .800
6         .800
7         .000
8         .833
9         .800
10        .571
11       1.000
12       1.000
13        .871
14        .962
15        .938
16        .750
17        .750
18       1.000
19        .722
20       1.000
21        .875
22    0.824125
23       1.000
24        .857
Name: stolenBasePercentage, dtype: object

In [59]:
df = df.apply(pd.to_numeric)

# Linear Regression to predict RBI

In [88]:
y = df["rbi"]
x = df.drop("rbi",axis=1)

### Normalize data

In [61]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_x = pd.DataFrame(x_scaled)
df_x.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0.691099,0.793651,0.938144,0.0,0.222222,0.507042,0.517241,0.55,0.461538,1.000000,...,0.291667,0.650307,0.491039,0.0,0.037037,1.00,0.461538,0.473684,0.0,0.120192
1,0.659686,0.277778,0.639175,0.0,0.888889,0.471831,0.172414,1.00,1.000000,0.918033,...,0.583333,0.263804,0.390681,0.0,0.037037,1.00,1.000000,0.934211,0.0,0.692308
2,0.324607,0.404762,0.412371,0.0,0.000000,0.887324,0.137931,0.25,0.115385,0.573770,...,0.541667,0.527607,0.526882,0.4,0.444444,0.75,0.115385,0.447368,0.0,0.530769
3,0.685864,0.642857,1.000000,0.0,0.222222,0.570423,0.655172,0.25,0.423077,0.622951,...,0.591667,0.613497,0.598566,0.5,0.185185,0.50,0.423077,0.644737,0.0,0.373077
4,0.115183,0.119048,0.474227,0.0,0.111111,0.239437,0.000000,0.10,0.192308,0.180328,...,0.000000,0.325153,0.175627,0.0,0.185185,1.00,0.192308,0.092105,0.0,0.175962


### Forward Selection

### Model Training

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_x, y, test_size = 0.3, random_state=101)

In [70]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [72]:
# Get all the beta coefficients
print(reg.coef_)

[  2.44846847   0.84679956   8.20347634   0.37038117  -2.23726644
  -1.01272495  -1.25901117  14.25202666  -2.54819781   4.39403129
  -4.41799244 -11.85886102   3.15177571   1.75987946   0.28894536
   5.33116234  -0.11124035  -5.23893475   0.88664117   1.67439778
  -0.1415787    3.3119328   -0.82678147  -1.86928639  -7.46360748
  -0.85002102  -0.26281609   0.1077704   -1.90203499   0.81430464
  10.71421231  -1.70041274  -6.28397142  -2.80304958   0.95343284
   1.41260319   3.08048413   3.84739455   4.33825325  -1.22688373
  -3.30412165  -0.51200023   1.80073741  -2.18259597  -0.44226894
   4.28206741   3.55265822   3.14698866  -2.4468279   -1.26077857
   0.           3.41874183]


In [73]:
reg.score(X_train,y_train)

1.0

In [90]:
y_test.reset_index(drop=True)

0    101
1    106
2     99
3    100
4    107
5     99
6    116
Name: rbi, dtype: int64

In [80]:
y_pred = reg.predict(X_test)

In [113]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

146.16704024028624
-3.41021242104312
